In [59]:
import pandas as pd
import numpy as np

In [60]:
dataa = pd.read_csv('data_with_prices.csv')

In [61]:
data = dataa.copy()

In [62]:
data.columns

Index(['ID', 'Country', 'VFN', 'Mp', 'Mh', 'Man', 'MMS', 'Tan', 'T', 'Va',
       'Ve', 'Mk', 'Cn', 'Ct', 'Cr', 'r', 'm (kg)', 'Mt', 'Enedc (g/km)',
       'Ewltp (g/km)', 'W (mm)', 'At1 (mm)', 'At2 (mm)', 'Ft', 'Fm',
       'ec (cm3)', 'ep (KW)', 'z (Wh/km)', 'IT', 'Ernedc (g/km)',
       'Erwltp (g/km)', 'De', 'Vf', 'Status', 'year', 'Date of registration',
       'Fuel consumption ', 'ech', 'RLFI', 'Electric range (km)', 'mk_norm',
       'cn_norm', 'price', 'frontal_area_m2'],
      dtype='object')

In [63]:
columns_to_dvs_and_hybrids = [ 'Mp',       # Производитель (для фильтрации)
    'Cn',       # Название модели (аналог 'name')
    'm (kg)',   # Масса (аналог 'mass_kg')
    'Ewltp (g/km)',  # Выбросы CO₂ (WLTP)
    'Fuel consumption ',  # Расход топлива (л/100км)
    'Ft',       # Тип топлива (petrol/diesel)
    'Fm',       # Режим работы (M=ДВС)
    'ep (KW)',  # Мощность двигателя (для расчёта КПД)
    'ec (cm3)',
     'price',
      "frontal_area_m2",
      'z (Wh/km)'
]

In [64]:
data_for_dvs_and_hybrids = data[columns_to_dvs_and_hybrids].copy()

# Добавляем расчётные поля:
data_for_dvs_and_hybrids['co2_emissions_gl'] = data_for_dvs_and_hybrids['Ewltp (g/km)'] * 100 / (data_for_dvs_and_hybrids['Fuel consumption '])
data_for_dvs_and_hybrids['engine_efficiency'] = np.where(
    data_for_dvs_and_hybrids['Ft'] == 'petrol', 
    0.25, 
    0.40
) 
data_for_dvs_and_hybrids['Energy per km (Wh/km)'] = (
    data_for_dvs_and_hybrids['z (Wh/km)'] * 0.1
)

In [65]:
data_for_dvs_and_hybrids.head()

,Mp,Cn,m (kg),Ewltp (g/km),Fuel consumption,Ft,Fm,ep (KW),ec (cm3),price,frontal_area_m2,z (Wh/km),co2_emissions_gl,engine_efficiency,Energy per km (Wh/km)
0,VOLKSWAGEN,CUPRA ATECA,1628.0,203.0,9.0,petrol,M,221.0,1984.0,40003.0,2.7,NaN,2255.555556,0.25,NaN
1,BMW,318D,1735.0,135.0,5.2,diesel,H,110.0,1995.0,11500.0,2.6,NaN,2596.153846,0.40,NaN
2,FORD,PUMA,1369.0,129.0,5.7,petrol,H,92.0,999.0,23158.0,2.8,NaN,2263.157895,0.25,NaN
3,VOLKSWAGEN,Q5,1880.0,176.0,6.7,diesel,H,150.0,1968.0,43193.0,3.1,NaN,2626.865672,0.40,NaN
4,VOLKSWAGEN,FORMENTOR E-HYBRID,1680.0,29.0,1.3,petrol/electric,P,110.0,1395.0,42454.0,2.5,153.0,2230.769231,0.40,15.3


In [66]:
data_for_dvs = data_for_dvs_and_hybrids[data_for_dvs_and_hybrids['Fm'] == 'M']
data_for_dvs

,Mp,Cn,m (kg),Ewltp (g/km),Fuel consumption,Ft,Fm,ep (KW),ec (cm3),price,frontal_area_m2,z (Wh/km),co2_emissions_gl,engine_efficiency,Energy per km (Wh/km)
0,VOLKSWAGEN,CUPRA ATECA,1628.0,203.0,9.0,petrol,M,221.0,1984.0,40003.0,2.7,NaN,2255.555556,0.25,NaN
7,VOLKSWAGEN,911 CARRERA T,1580.0,240.0,10.6,petrol,M,283.0,2981.0,108136.0,2.8,NaN,2264.150943,0.25,NaN
8,VOLKSWAGEN,Q3,1695.0,188.0,8.3,petrol,M,140.0,1984.0,43820.0,2.5,NaN,2265.060241,0.25,NaN
10,SUBARU-SUZUKI-TOYOTA,TOYOTA AYGO X,1025.0,109.0,4.8,petrol,M,53.0,998.0,8636.0,2.5,NaN,2270.833333,0.25,NaN
12,BMW,COOPER S,1400.0,143.0,6.3,petrol,M,131.0,1998.0,15575.0,2.7,NaN,2269.841270,0.25,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786,VOLKSWAGEN,A1 SPORTBA,1205.0,133.0,5.9,petrol,M,81.0,999.0,13010.0,2.5,NaN,2254.237288,0.25,NaN
1787,VOLKSWAGEN,A3 SPORTBA,1295.0,125.0,NaN,petrol,M,81.0,999.0,16869.0,2.4,NaN,NaN,0.25,NaN
1788,BMW,X2 XDRIVE25D,1695.0,153.0,NaN,diesel,M,170.0,1995.0,32215.0,2.7,NaN,NaN,0.40,NaN
1791,SUBARU-SUZUKI-TOYOTA,IMPREZA XV,1502.0,180.0,8.0,petrol,M,84.0,1600.0,12647.0,2.6,NaN,2250.000000,0.25,NaN


In [67]:
data_for_dvs = data_for_dvs.drop(columns=['Ewltp (g/km)', 'Ft','Fm','ep (KW)', 'ec (cm3)', 'z (Wh/km)', 'Energy per km (Wh/km)' ]) 
data_for_dvs

,Mp,Cn,m (kg),Fuel consumption,price,frontal_area_m2,co2_emissions_gl,engine_efficiency
0,VOLKSWAGEN,CUPRA ATECA,1628.0,9.0,40003.0,2.7,2255.555556,0.25
7,VOLKSWAGEN,911 CARRERA T,1580.0,10.6,108136.0,2.8,2264.150943,0.25
8,VOLKSWAGEN,Q3,1695.0,8.3,43820.0,2.5,2265.060241,0.25
10,SUBARU-SUZUKI-TOYOTA,TOYOTA AYGO X,1025.0,4.8,8636.0,2.5,2270.833333,0.25
12,BMW,COOPER S,1400.0,6.3,15575.0,2.7,2269.841270,0.25
...,...,...,...,...,...,...,...,...
1786,VOLKSWAGEN,A1 SPORTBA,1205.0,5.9,13010.0,2.5,2254.237288,0.25
1787,VOLKSWAGEN,A3 SPORTBA,1295.0,NaN,16869.0,2.4,NaN,0.25
1788,BMW,X2 XDRIVE25D,1695.0,NaN,32215.0,2.7,NaN,0.40
1791,SUBARU-SUZUKI-TOYOTA,IMPREZA XV,1502.0,8.0,12647.0,2.6,2250.000000,0.25


In [68]:
data_for_dvs.columns = ['mark_name', 'model_name', 'mass_kg', 'fuel_consumption_lp100km', 'production_price',"frontal_area_m2", 'co2_emissions_gl', 'engine_efficiency'] 
data_for_dvs

,mark_name,model_name,mass_kg,fuel_consumption_lp100km,production_price,frontal_area_m2,co2_emissions_gl,engine_efficiency
0,VOLKSWAGEN,CUPRA ATECA,1628.0,9.0,40003.0,2.7,2255.555556,0.25
7,VOLKSWAGEN,911 CARRERA T,1580.0,10.6,108136.0,2.8,2264.150943,0.25
8,VOLKSWAGEN,Q3,1695.0,8.3,43820.0,2.5,2265.060241,0.25
10,SUBARU-SUZUKI-TOYOTA,TOYOTA AYGO X,1025.0,4.8,8636.0,2.5,2270.833333,0.25
12,BMW,COOPER S,1400.0,6.3,15575.0,2.7,2269.841270,0.25
...,...,...,...,...,...,...,...,...
1786,VOLKSWAGEN,A1 SPORTBA,1205.0,5.9,13010.0,2.5,2254.237288,0.25
1787,VOLKSWAGEN,A3 SPORTBA,1295.0,NaN,16869.0,2.4,NaN,0.25
1788,BMW,X2 XDRIVE25D,1695.0,NaN,32215.0,2.7,NaN,0.40
1791,SUBARU-SUZUKI-TOYOTA,IMPREZA XV,1502.0,8.0,12647.0,2.6,2250.000000,0.25


In [69]:
data_for_dvs.to_csv('data_for_dvs.csv', index=False)

In [70]:
data_for_hibrids = data_for_dvs_and_hybrids[data_for_dvs_and_hybrids['Fm'] == 'H']
data_for_hibrids

,Mp,Cn,m (kg),Ewltp (g/km),Fuel consumption,Ft,Fm,ep (KW),ec (cm3),price,frontal_area_m2,z (Wh/km),co2_emissions_gl,engine_efficiency,Energy per km (Wh/km)
1,BMW,318D,1735.0,135.0,5.2,diesel,H,110.0,1995.0,11500.0,2.6,NaN,2596.153846,0.40,NaN
2,FORD,PUMA,1369.0,129.0,5.7,petrol,H,92.0,999.0,23158.0,2.8,NaN,2263.157895,0.25,NaN
3,VOLKSWAGEN,Q5,1880.0,176.0,6.7,diesel,H,150.0,1968.0,43193.0,3.1,NaN,2626.865672,0.40,NaN
11,HYUNDAI MOTOR EUROPE,BAYON,1220.0,119.0,5.2,petrol,H,74.0,998.0,22990.0,2.5,NaN,2288.461538,0.25,NaN
13,RENAULT-NISSAN-MITSUBISHI,CAPTUR,1373.0,135.0,6.0,petrol,H,103.0,1332.0,15051.0,2.0,NaN,2250.000000,0.25,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1763,BMW,X5 XDRIVE 40 I,2240.0,202.0,9.0,petrol,H,280.0,2998.0,40997.0,2.1,NaN,2244.444444,0.25,NaN
1767,VOLKSWAGEN,A5 COUPC‰,1655.0,149.0,5.7,diesel,H,150.0,1968.0,28690.0,2.5,NaN,2614.035088,0.40,NaN
1768,VOLKSWAGEN,S5 COUPC‰,1860.0,190.0,7.2,diesel,H,251.0,2967.0,49735.0,2.6,NaN,2638.888889,0.40,NaN
1789,BMW,X1 SDRIVE2,1625.0,137.0,6.1,petrol,H,115.0,1499.0,18271.0,3.0,NaN,2245.901639,0.25,NaN


In [71]:
data_for_hibrids = data_for_hibrids.drop(columns=['Ewltp (g/km)', 'Ft','Fm','ep (KW)', 'ec (cm3)', 'z (Wh/km)', 'Energy per km (Wh/km)' ]) 
data_for_hibrids.columns = ['mark_name', 'model_name', 'mass_kg', 'fuel_consumption_lp100km', 'production_price', "frontal_area_m2",'co2_emissions_gl', 'engine_efficiency'] 
data_for_hibrids.to_csv('data_for_hibrids.csv', index=False)
data_for_hibrids

,mark_name,model_name,mass_kg,fuel_consumption_lp100km,production_price,frontal_area_m2,co2_emissions_gl,engine_efficiency
1,BMW,318D,1735.0,5.2,11500.0,2.6,2596.153846,0.40
2,FORD,PUMA,1369.0,5.7,23158.0,2.8,2263.157895,0.25
3,VOLKSWAGEN,Q5,1880.0,6.7,43193.0,3.1,2626.865672,0.40
11,HYUNDAI MOTOR EUROPE,BAYON,1220.0,5.2,22990.0,2.5,2288.461538,0.25
13,RENAULT-NISSAN-MITSUBISHI,CAPTUR,1373.0,6.0,15051.0,2.0,2250.000000,0.25
...,...,...,...,...,...,...,...,...
1763,BMW,X5 XDRIVE 40 I,2240.0,9.0,40997.0,2.1,2244.444444,0.25
1767,VOLKSWAGEN,A5 COUPC‰,1655.0,5.7,28690.0,2.5,2614.035088,0.40
1768,VOLKSWAGEN,S5 COUPC‰,1860.0,7.2,49735.0,2.6,2638.888889,0.40
1789,BMW,X1 SDRIVE2,1625.0,6.1,18271.0,3.0,2245.901639,0.25


In [72]:
columns_for_electric = ['Mp',       # Производитель (для фильтрации)
    'Cn',       # Название модели
    'Fm',             # Тип привода (E/H)
    'z (Wh/km)',  # Потребление энергии (кВт·ч/100 км)
    'Electric range (km)',   # Запас хода
    'm (kg)',
    'price',
    "frontal_area_m2"
]
data_for_bev = data[data['Fm'] == 'E'].copy()

In [73]:

data_for_bev = data_for_bev[columns_for_electric]

# Шаг 3: Добавляем расчётные колонки
# 1. Ёмкость аккумулятора (кВт·ч)
data_for_bev['Battery capacity (kWh)'] = (
    data_for_bev['z (Wh/km)'] * 
    data_for_bev['Electric range (km)'] / 100
)

# 2. Удельное потребление энергии (кВт·ч/100км)
data_for_bev['Energy per km (Wh/km)'] = (
    data_for_bev['z (Wh/km)'] / 10
)

# 3. Предполагаемый КПД электродвигателя (90%)
data_for_bev['Motor efficiency'] = 0.90

# 4. Энергоэффективность (км/кВт·ч)
data_for_bev['Efficiency (km/kWh)'] = (
    100 / data_for_bev['z (Wh/km)']
)

In [74]:
data_for_bev

,Mp,Cn,Fm,z (Wh/km),Electric range (km),m (kg),price,frontal_area_m2,Battery capacity (kWh),Energy per km (Wh/km),Motor efficiency,Efficiency (km/kWh)
5,MAZDA,MAZDA MX-30,E,179.0,200.0,1720.0,19985.0,2.5,358.00,17.9,0.9,0.558659
6,VOLKSWAGEN,Q8 SPORTBACK 50 E-TRON,E,217.0,455.0,2585.0,89275.0,3.1,987.35,21.7,0.9,0.460829
9,STELLANTIS,CORSA,E,160.0,351.0,1530.0,8959.0,2.8,561.60,16.0,0.9,0.625000
24,RENAULT-NISSAN-MITSUBISHI,TWINGO,E,152.0,187.0,1208.0,7478.0,2.4,284.24,15.2,0.9,0.657895
33,VOLKSWAGEN,ENYAQ RS,E,172.0,508.0,2253.0,48642.0,2.3,873.76,17.2,0.9,0.581395
...,...,...,...,...,...,...,...,...,...,...,...,...
1747,STELLANTIS,3 CROSSBACK,E,161.0,330.0,1598.0,25223.0,2.8,531.30,16.1,0.9,0.621118
1753,BMW,IX2 XDRIVE30,E,174.0,420.0,2095.0,32215.0,2.1,730.80,17.4,0.9,0.574713
1771,STELLANTIS,ABARTH 500E,E,180.0,254.0,1410.0,22508.0,3.1,457.20,18.0,0.9,0.555556
1780,STELLANTIS,JUMPY I SP,E,270.0,313.0,2161.0,11144.0,2.2,845.10,27.0,0.9,0.370370


In [75]:
data_for_bev = data_for_bev.drop(columns=['Fm','z (Wh/km)', 'Electric range (km)']) 
data_for_bev

,Mp,Cn,m (kg),price,frontal_area_m2,Battery capacity (kWh),Energy per km (Wh/km),Motor efficiency,Efficiency (km/kWh)
5,MAZDA,MAZDA MX-30,1720.0,19985.0,2.5,358.00,17.9,0.9,0.558659
6,VOLKSWAGEN,Q8 SPORTBACK 50 E-TRON,2585.0,89275.0,3.1,987.35,21.7,0.9,0.460829
9,STELLANTIS,CORSA,1530.0,8959.0,2.8,561.60,16.0,0.9,0.625000
24,RENAULT-NISSAN-MITSUBISHI,TWINGO,1208.0,7478.0,2.4,284.24,15.2,0.9,0.657895
33,VOLKSWAGEN,ENYAQ RS,2253.0,48642.0,2.3,873.76,17.2,0.9,0.581395
...,...,...,...,...,...,...,...,...,...
1747,STELLANTIS,3 CROSSBACK,1598.0,25223.0,2.8,531.30,16.1,0.9,0.621118
1753,BMW,IX2 XDRIVE30,2095.0,32215.0,2.1,730.80,17.4,0.9,0.574713
1771,STELLANTIS,ABARTH 500E,1410.0,22508.0,3.1,457.20,18.0,0.9,0.555556
1780,STELLANTIS,JUMPY I SP,2161.0,11144.0,2.2,845.10,27.0,0.9,0.370370


In [76]:
data_for_bev.columns = ['mark_name', 'model_name', 'mass_kg', 'price',"frontal_area_m2", 'Battery capacity (kWh)', 'Energy per km (Wh/km)', 'Motor efficiency', 'Efficiency (km/kWh)'] 
data_for_bev.to_csv('data_for_bev.csv', index=False)
data_for_bev

,mark_name,model_name,mass_kg,price,frontal_area_m2,Battery capacity (kWh),Energy per km (Wh/km),Motor efficiency,Efficiency (km/kWh)
5,MAZDA,MAZDA MX-30,1720.0,19985.0,2.5,358.00,17.9,0.9,0.558659
6,VOLKSWAGEN,Q8 SPORTBACK 50 E-TRON,2585.0,89275.0,3.1,987.35,21.7,0.9,0.460829
9,STELLANTIS,CORSA,1530.0,8959.0,2.8,561.60,16.0,0.9,0.625000
24,RENAULT-NISSAN-MITSUBISHI,TWINGO,1208.0,7478.0,2.4,284.24,15.2,0.9,0.657895
33,VOLKSWAGEN,ENYAQ RS,2253.0,48642.0,2.3,873.76,17.2,0.9,0.581395
...,...,...,...,...,...,...,...,...,...
1747,STELLANTIS,3 CROSSBACK,1598.0,25223.0,2.8,531.30,16.1,0.9,0.621118
1753,BMW,IX2 XDRIVE30,2095.0,32215.0,2.1,730.80,17.4,0.9,0.574713
1771,STELLANTIS,ABARTH 500E,1410.0,22508.0,3.1,457.20,18.0,0.9,0.555556
1780,STELLANTIS,JUMPY I SP,2161.0,11144.0,2.2,845.10,27.0,0.9,0.370370
